In [2]:
from azureml.core import Workspace

# Create or retrieve an existing Azure ML workspace
ws = Workspace.get(name='WorkspaceOne',
                      subscription_id='0f40667b-82a1-49fb-92ed-e02acd56126b',
                      resource_group='nolan_bryson-rg',
                      location='westus')


# Write configuration to the workspace config file
ws.write_config(path='.azureml')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network with one hidden layer
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # Input layer (784 input features)
        self.fc2 = nn.Linear(128, 10)   # Output layer (10 classes)
        self.relu = nn.ReLU()           # Activation function

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Apply ReLU after the first layer
        x = self.fc2(x)             # Output layer
        return x

# Initialize the neural network
model = SimpleNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # For classification tasks
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

In [4]:
from torchvision import datasets, transforms

# Load MNIST dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=32, shuffle=True)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Reset gradients

        # Forward pass
        outputs = model(inputs.view(-1, 784))  # Flatten input
        loss = criterion(outputs, labels)      # Compute loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

Epoch 1, Loss: 0.8859738236665726
Epoch 2, Loss: 0.3790428863366445
Epoch 3, Loss: 0.3254845398545265
Epoch 4, Loss: 0.2963150965730349
Epoch 5, Loss: 0.2731281920870145


In [5]:
from azureml.core import Model

# Save the trained model
torch.save(model.state_dict(), 'simple_nn.pth')

# Register the model in Azure
model = Model.register(workspace=ws, model_path='simple_nn.pth', model_name='simple_nn')

# Deploying as a web service requires more steps such as creating a scoring script and environment.

Registering model simple_nn
